In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import corner
from astropy.io import fits
from astropy.table import Table
from ParamVAEapply import ParamVAEapply
from BandPassVAE import BandPassVAE
from SpectraVAE import SpectraVAE
from ParamVAE import ParamVAE
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from datetime import datetime


2025-11-14 14:02:36.977788: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-14 14:02:36.985334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763125356.993980   12380 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763125356.996607   12380 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763125357.003298   12380 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [6]:
def load_paramvaeapply(path: str, weight_file: str, input_dim: int, latent_dim: int, beta: float) -> ParamVAEapply:
    """
    Loads weights and builds a ParamVAEapply model from a saved epoch weight file.

    Parameters:
        weight_file (str): Path to the weights file (.h5).
        input_dim (int): Dimensionality of the input data.
        latent_dim (int): Latent space dimensionality.
        beta (float): Beta value for VAE loss.

    Returns:
        ParamVAEapply: A ready-to-use model with loaded weights.
    """
    # Build required submodels
    
    spvae_wf = os.path.join(os.path.join(path, 'SpectraVAE'), weight_file)
    bpvae_wf = os.path.join(os.path.join(path, 'BandPassVAE'), weight_file)
    parvae_wf = os.path.join(os.path.join(path, 'ParamVAE'), weight_file)
    print(spvae_wf, bpvae_wf, parvae_wf)
    spvae = SpectraVAE(100, latent_dim, beta=beta)
    spvae.compile(optimizer=keras.optimizers.Adam())
    spvae(tf.zeros((1, 100)))  # Force model building
    spvae.load_weights(spvae_wf)

    bpvae = BandPassVAE(input_dim, latent_dim, spvae, beta=beta)
    bpvae.compile(optimizer=keras.optimizers.Adam())
    bpvae(tf.zeros((1, input_dim)))  # Force model building
    bpvae.load_weights(bpvae_wf)

    # Attach ParamVAE
    parvae = ParamVAE(100, latent_dim, spvae, beta=beta, n_param=4)
    parvae.compile(optimizer=keras.optimizers.Adam())
    parvae(tf.zeros((1, 100)))  # Force model building
    parvae.load_weights(parvae_wf)

    return ParamVAEapply(input_dim, latent_dim, bpvae, parvae, beta=beta), parvae, bpvae, spvae


In [7]:
# Configuration
latent_dim = 64
beta = 1e-3
epochs = 150
n_param = 4
input_dim = 5  # Number of input bands


id_run = '20251010162214'
#id_run = '20250910142657'
weight_path = "/data/kirg/MMVAE/cigale/"+id_run+"/"
save_path = os.path.join(weight_path, 'epochs/')
plot_output_path = os.path.join(weight_path, 'cornerplots_train_'+id_run)


epoch = 150
weight_file = f'weights_epoch_{epoch:02d}.weights.h5'

parvaeapply, parvae, bpvae, spvae = load_paramvaeapply(save_path, weight_file, input_dim, latent_dim, beta)


/data/kirg/MMVAE/cigale/20251010162214/epochs/SpectraVAE/weights_epoch_150.weights.h5 /data/kirg/MMVAE/cigale/20251010162214/epochs/BandPassVAE/weights_epoch_150.weights.h5 /data/kirg/MMVAE/cigale/20251010162214/epochs/ParamVAE/weights_epoch_150.weights.h5


I0000 00:00:1763125358.086468   12380 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22444 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6
/home/kirg/envs/envp3.12/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(1, 100))
  warnings.warn(msg)
/home/kirg/envs/envp3.12/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/kirg/envs/envp3.12/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_16']
Received: inputs=Tensor

In [8]:
fits_file = 'HSC_SSP_photcat.fits'
t_hsc_ = Table.read(fits_file)


In [105]:
cen_ra = 39.2721 #34.2335#33.8216+0.0
cen_dec = -3.9546 #-0.0851#0.0154-0.0
t_hsc = t_hsc_ #[(np.abs(t_hsc_['ra'] - cen_ra) + np.abs(t_hsc_['dec'] - cen_dec)) < 240./60.]

In [106]:
hsc_int = []
mask = [False] * len(t_hsc)
for i,row in enumerate(t_hsc):
    mags = np.array([
        row['g_kronflux_mag'],
        row['r_kronflux_mag'],
        row['i_kronflux_mag'],
        row['z_kronflux_mag'],
        row['y_kronflux_mag']
    ])
    band_flux = 10**(-0.4 * mags)
    band_flux /= np.max(band_flux)
    if np.isfinite(np.max(band_flux)):
        hsc_int.append(band_flux)
        mask[i] = True


hsc_input = np.reshape(hsc_int, (len(hsc_int), input_dim))


/tmp/ipykernel_12380/1612501144.py:4: UserWarning: Warning: converting a masked element to nan.
  mags = np.array([


In [107]:
print(hsc_input)

[[0.42179006 0.6057299  0.4887867  0.5406211  1.        ]
 [0.13184069 0.35395128 0.92227805 1.         0.3212976 ]
 [0.7357265  0.7787627  1.         0.96353954 0.9010861 ]
 ...
 [0.12221295 0.06437932 0.12506795 0.61705416 1.        ]
 [0.10572938 0.38092974 0.7791356  0.64959127 1.        ]
 [0.5144484  0.6439458  0.78716487 1.         0.7751763 ]]


In [108]:
n_smpl = 10

n = len(hsc_int)

s_all = []
for i in range(n_smpl):
    s = parvaeapply(np.reshape(hsc_int, (n,5)))
    s_all.append(s)


/home/kirg/envs/envp3.12/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_16']
Received: inputs=Tensor(shape=(31837003, 5))
  warnings.warn(msg)
2025-11-14 16:11:30.260875: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.59GiB (rounded to 8150272768)requested by op MatMul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-11-14 16:11:30.260929: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-11-14 16:11:30.260948: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 102, Chunks in use: 101. 25.5KiB allocated for chunks. 25.2KiB in use in bin. 4.9KiB client

ResourceExhaustedError: Exception encountered when calling Dense.call().

[1m{{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[31837003,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:MatMul] name: [0m

Arguments received by Dense.call():
  • inputs=tf.Tensor(shape=(31837003, 16), dtype=float32)
  • training=True

In [ ]:
s_all = np.array(s_all)
s_all_mean = np.nanmean(s_all[:, :, 0], axis=0)

zphoto = s_all[:, :, 0].flatten()

#plt.hist(s_all[:, :, 0].flatten(), bins=100)
plt.hist(s_all_mean, bins=100)
plt.xlim(0.5, 0.75)
plt.show()

print(s_all[:, :, 0].flatten().shape)

#t_sel = t_hsc[mask][(s_all_mean > 0.50) & (s_all_mean < 0.65)]
t_sel = t_hsc[mask][(s_all_mean > 0.15) & (s_all_mean < 0.20)]

print(t_sel['ra', 'dec', 'g_kronflux_mag', 'r_kronflux_mag', 'i_kronflux_mag', 'z_kronflux_mag'])

#plt.scatter(t_hsc[mask]['ra'], t_hsc[mask]['dec'], s=1)
#plt.scatter(t_sel['ra'], t_sel['dec'], s=1) #, c=s_all_mean)


plt.hist2d(t_sel['ra'], t_sel['dec'], bins=200)
#plt.xlim(36.0, 37)
#plt.ylim(-3, -2.)
plt.show()

In [ ]:
from ipyaladin import Aladin
aladin = Aladin()
aladin